In [265]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

In [266]:
#Get relative path of the CSV File 
#Enumerate therough all the files in the folder and check if the path exists

mypath = os.path.dirname(os.path.realpath('train.csv'))
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
os.path.exists(mypath)

True

In [267]:
print(onlyfiles[4])

train.csv


In [268]:
#Read Training CSV for house regression
dataset = pd.read_csv(onlyfiles[4])
dataset.shape

(1460, 81)

In [269]:
testDataset = pd.read_csv(onlyfiles[3])
testDataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


importing the dataset 
encode categorical data
Backward elimination

In [270]:
#Delete redundant features with over 50% NAN values E.G: porch sizes, pool area, alley, miscallenous features and its value 
unwantedFeaturesList = ["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea",
                       "PoolQC","Fence","Alley","MiscFeature","MiscVal"]
dataset.drop(unwantedFeaturesList,inplace=True,axis=1)
testDataset.drop(unwantedFeaturesList,inplace=True,axis=1)

In [271]:
# Check all the columns fetures whcih has a numerical value type
is_number = np.vectorize(lambda x: np.issubdtype(x, np.number))
is_number(dataset.dtypes)

array([ True,  True, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True,  True, False, False, False, False, False,  True, False,
       False, False, False, False, False, False,  True, False,  True,
        True,  True, False, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
       False,  True, False, False,  True, False,  True,  True, False,
       False, False,  True,  True, False, False,  True], dtype=bool)

In [272]:
#Separate and select dataframes by dataTypes 
df_NonNumericalType = dataset.select_dtypes(exclude=[np.number])
df_NumericalType = dataset.select_dtypes(include=[np.number])
df_TestNonNumericalType = testDataset.select_dtypes(exclude=[np.number])
df_TestNumericalType = testDataset.select_dtypes(include=[np.number])
#Set type to categorical
df_NonNumericalType[df_NonNumericalType.select_dtypes(['object']).columns] = df_NonNumericalType.select_dtypes(['object']).apply(lambda x: x.astype('category'))
df_TestNonNumericalType[df_TestNonNumericalType.select_dtypes(['object']).columns] = df_TestNonNumericalType.select_dtypes(['object']).apply(lambda x: x.astype('category'))
df_NonNumericalType.dtypes

C:\Users\BOBBOO\Anaconda3\lib\site-packages\pandas\core\frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


MSZoning         category
Street           category
LotShape         category
LandContour      category
Utilities        category
LotConfig        category
LandSlope        category
Neighborhood     category
Condition1       category
Condition2       category
BldgType         category
HouseStyle       category
RoofStyle        category
RoofMatl         category
Exterior1st      category
Exterior2nd      category
MasVnrType       category
ExterQual        category
ExterCond        category
Foundation       category
BsmtQual         category
BsmtCond         category
BsmtExposure     category
BsmtFinType1     category
BsmtFinType2     category
Heating          category
HeatingQC        category
CentralAir       category
Electrical       category
KitchenQual      category
Functional       category
FireplaceQu      category
GarageType       category
GarageFinish     category
GarageQual       category
GarageCond       category
PavedDrive       category
SaleType         category
SaleConditio

In [273]:
df_NumericalType.drop(["SalePrice"], inplace=True, axis=1)
#replace all the NA values to numpy NAN for float 64 data type
df_NumericalType.replace('NA',np.nan)
df_TestNumericalType.replace('NA',np.nan)
#Select Numerical values except the index 
xNumerical = df_NumericalType.values
xTestNumerical = df_TestNumericalType.values
#drop the SalesPrice column


C:\Users\BOBBOO\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [274]:
print(len(df_NumericalType.columns))

30


In [275]:
#Handle missing data for numerical values 
from sklearn.preprocessing import Imputer
if df_NumericalType.isnull().values.any() :
    #mean is the deafualt parameter anywyas
    imputer = Imputer(missing_values = 'NaN', strategy='mean', axis=0)
    imputer.fit(xTestNumerical[:,2:32])
    xNumerical[:,2:32] = imputer.transform(xNumerical[:,2:32])

In [276]:
df_NumericalType.shape, xNumerical.shape

((1460, 30), (1460, 30))

In [277]:
#Handle missing data for numerical values 
from sklearn.preprocessing import Imputer
if df_TestNumericalType.isnull().values.any() :
    #mean is the deafualt parameter anywyas
    imputer = Imputer(missing_values = 'NaN', strategy='mean', axis=0)
    imputer.fit(xTestNumerical[:,2:32])
    xTestNumerical[:,2:32] = imputer.transform(xTestNumerical[:,2:32])

In [278]:
#Scaling Numerical data
from sklearn.preprocessing import StandardScaler
sc_X= StandardScaler()
NumericalScaledData = sc_X.fit_transform(xNumerical)
TestNumericalScaledData = sc_X.fit_transform(xTestNumerical)

In [313]:
df_AllNonNumerical = pd.concat([df_NonNumericalType, df_TestNonNumericalType], axis=0)
df_AllNonNumerical[df_AllNonNumerical.select_dtypes(['object']).columns] = df_AllNonNumerical.select_dtypes(['object']).apply(lambda x: x.astype('category'))
cat_features = [key for key in dict(df_AllNonNumerical.dtypes) if dict(df_AllNonNumerical.dtypes)[key] in ['category']]
df_AllNonNumericalEncoded = pd.get_dummies(df_AllNonNumerical, columns=cat_features, drop_first=True)
df_NonNumericalencoded = df_AllNonNumericalEncoded[:len(df_NonNumericalType.values)]
df_TestNonNumericalencoded = df_AllNonNumericalEncoded[len(df_NonNumericalType.values):]
df_AllNonNumerical.dtypes

MSZoning         category
Street           category
LotShape         category
LandContour      category
Utilities        category
LotConfig        category
LandSlope        category
Neighborhood     category
Condition1       category
Condition2       category
BldgType         category
HouseStyle       category
RoofStyle        category
RoofMatl         category
Exterior1st      category
Exterior2nd      category
MasVnrType       category
ExterQual        category
ExterCond        category
Foundation       category
BsmtQual         category
BsmtCond         category
BsmtExposure     category
BsmtFinType1     category
BsmtFinType2     category
Heating          category
HeatingQC        category
CentralAir       category
Electrical       category
KitchenQual      category
Functional       category
FireplaceQu      category
GarageType       category
GarageFinish     category
GarageQual       category
GarageCond       category
PavedDrive       category
SaleType         category
SaleConditio

In [300]:
#Sort by categorical data and encoding it
#Create dummy data for all the categorical columns
cat_features = [key for key in dict(df_NonNumericalType.dtypes) if dict(df_NonNumericalType.dtypes)[key] in ['category']]
df_NonNumericalencoded = pd.get_dummies(df_NonNumericalType, columns=cat_features, drop_first=True)
#df_NonNumericalencoded.reset_index(level=0, inplace=True)
#df_NumericalType.reset_index(level=0, inplace=True)

In [301]:
#Sort by categorical data and encoding it
#Create dummy data for all the categorical columns
#cat_features = [key for key in dict(df_TestNonNumericalType.dtypes) if dict(df_TestNonNumericalType.dtypes)[key] in ['category']]
df_TestNonNumericalencoded = pd.get_dummies(df_TestNonNumericalType, columns=cat_features, drop_first=True)
#df_TestNonNumericalencoded.reset_index(level=0, inplace=True)
#df_TestNumericalType.reset_index(level=0, inplace=True)

In [302]:
df_NonNumericalencoded.columns, df_TestNonNumericalencoded.columns

(Index(['MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM',
        'Street_Pave', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg',
        'LandContour_HLS', 'LandContour_Low',
        ...
        'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth',
        'SaleType_WD', 'SaleCondition_AdjLand', 'SaleCondition_Alloca',
        'SaleCondition_Family', 'SaleCondition_Normal',
        'SaleCondition_Partial'],
       dtype='object', length=200),
 Index(['MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM',
        'Street_Pave', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg',
        'LandContour_HLS', 'LandContour_Low',
        ...
        'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth',
        'SaleType_WD', 'SaleCondition_AdjLand', 'SaleCondition_Alloca',
        'SaleCondition_Family', 'SaleCondition_Normal',
        'SaleCondition_Partial'],
       dtype='object', length=184))

In [281]:
df_NumericalType.columns

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'MoSold',
       'YrSold'],
      dtype='object')

In [314]:
df_NumericalScaled = pd.DataFrame(NumericalScaledData)
df_NumericalScaled.columns = df_NumericalType.columns
df_AllTypes = pd.concat([df_NumericalScaled, df_NonNumericalencoded], axis=1)

In [284]:
TestNumericalScaledData.shape, df_TestNumericalType.shape

((1459, 30), (1459, 30))

In [315]:
df_TestNumericalScaled = pd.DataFrame(TestNumericalScaledData)
df_TestNumericalScaled.columns = df_TestNumericalType.columns
df_AllTestTypes = pd.concat([df_TestNumericalScaled, df_TestNonNumericalencoded], axis=1)

In [316]:
from sklearn.tree import *
df_Y = dataset["SalePrice"]

In [317]:
df_AllTypes.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-1.730865,0.073375,-0.217460,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.511508,0.575425,...,0,0,0,0,1,0,0,0,1,0
1,-1.728492,-0.872563,0.463627,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.574320,1.171992,...,0,0,0,0,1,0,0,0,1,0
2,-1.726120,0.073375,-0.081242,0.073480,0.651479,-0.517200,0.984752,0.830215,0.323150,0.092907,...,0,0,0,0,1,0,0,0,1,0
3,-1.723747,0.309859,-0.444489,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.574320,-0.499274,...,0,0,0,0,1,0,0,0,0,0
4,-1.721374,0.073375,0.645250,0.375148,1.374795,-0.517200,0.951632,0.733308,1.364659,0.463568,...,0,0,0,0,1,0,0,0,1,0


In [318]:
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split

In [319]:
df_AllTestTypes.shape, df_Y.shape

((1459, 230), (1460,))

In [320]:
reg = DecisionTreeRegressor()
reg.fit(df_AllTypes.values, df_Y.values)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [332]:
df_AllTestTypes.shape, df_AllTypes.shape
df_Y.drop(df_Y.index[0], inplace=True)

In [333]:
reg.predict(df_AllTestTypes.values)
reg.score(df_AllTestTypes.values, df_Y.values)

-0.94247758924844383

In [151]:
df_AllTypes.head()

,index,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-1.730865,0.073375,-0.229372,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.511418,0.575425,...,0,0,0,0,1,0,0,0,1,0
1,-1.728492,-0.872563,0.451936,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.574410,1.171992,...,0,0,0,0,1,0,0,0,1,0
2,-1.726120,0.073375,-0.093110,0.073480,0.651479,-0.517200,0.984752,0.830215,0.323060,0.092907,...,0,0,0,0,1,0,0,0,1,0
3,-1.723747,0.309859,-0.456474,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.574410,-0.499274,...,0,0,0,0,1,0,0,0,0,0
4,-1.721374,0.073375,0.633618,0.375148,1.374795,-0.517200,0.951632,0.733308,1.364570,0.463568,...,0,0,0,0,1,0,0,0,1,0
